# Evaluación del impacto de un Agente de IA en la eficiencia empresarial mediante técnicas de aprendizaje automático

## Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from scipy import stats
from scipy.stats import pearsonr, spearmanr, shapiro
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

## Carga y Preparación de Datos

In [3]:
from google.colab import files
import pandas as pd

# Subir archivo
uploaded = files.upload()

Saving Evaluación_de_Agente_de_IA[1].csv to Evaluación_de_Agente_de_IA[1].csv


In [4]:
df = pd.read_csv('Evaluación_de_Agente_de_IA[1].csv')

# Renombrar columnas para facilitar el manejo
df.columns = [
    'timestamp', 'rol', 'frecuencia_uso', 'metodo_anterior', 'tiempo_sin_ia',
    'tiempo_con_ia', 'satisfaccion_general', 'facilidad_uso', 'eficacia',
    'claridad', 'gusto', 'desafios', 'mejoras', 'impacto_productividad',
    'otros_comentarios'
]

# Convertir columnas numéricas con validación mejorada
numeric_cols = ['tiempo_sin_ia', 'tiempo_con_ia', 'satisfaccion_general', 'facilidad_uso', 'eficacia', 'claridad']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Variables derivadas con validación
df['ahorro_tiempo'] = df['tiempo_sin_ia'] - df['tiempo_con_ia']
df['porcentaje_ahorro'] = (df['ahorro_tiempo'] / df['tiempo_sin_ia']) * 100
df['eficiencia_relativa'] = df['tiempo_sin_ia'] / df['tiempo_con_ia']
df['satisfaccion_total'] = df[['satisfaccion_general', 'facilidad_uso', 'eficacia', 'claridad']].mean(axis=1)

print(f"📊 DATASET PROCESADO: {len(df)} observaciones")
print(f"Variables clave generadas: ahorro_tiempo, porcentaje_ahorro, satisfaccion_total")

📊 DATASET PROCESADO: 8 observaciones
Variables clave generadas: ahorro_tiempo, porcentaje_ahorro, satisfaccion_total


## Análisis estadistico

In [5]:
print("\n=== ANÁLISIS ESTADÍSTICO ROBUSTO ===")

# Función mejorada para bootstrap con intervalos de confianza
def robust_bootstrap_ci(data, statistic_func=np.mean, n_bootstrap=10000, confidence_level=0.95):
    """Calcula intervalos de confianza bootstrap más robustos"""
    if len(data) < 3:
        return None, None, None

    np.random.seed(42)
    bootstrap_stats = []

    for _ in range(n_bootstrap):
        bootstrap_sample = np.random.choice(data, size=len(data), replace=True)
        bootstrap_stats.append(statistic_func(bootstrap_sample))

    alpha = 1 - confidence_level
    ci_lower = np.percentile(bootstrap_stats, (alpha/2) * 100)
    ci_upper = np.percentile(bootstrap_stats, (1 - alpha/2) * 100)

    return bootstrap_stats, ci_lower, ci_upper

# Métricas clave con intervalos de confianza
key_metrics = {
    'Tiempo Sin IA (min)': df['tiempo_sin_ia'].dropna(),
    'Tiempo Con IA (min)': df['tiempo_con_ia'].dropna(),
    'Ahorro de Tiempo (min)': df['ahorro_tiempo'].dropna(),
    'Ahorro Porcentual (%)': df['porcentaje_ahorro'].dropna(),
    'Satisfacción Total': df['satisfaccion_total'].dropna(),
    'Eficacia': df['eficacia'].dropna(),
    'Facilidad de Uso': df['facilidad_uso'].dropna()
}

print("\n📊 MÉTRICAS PRINCIPALES CON INTERVALOS DE CONFIANZA (95%):")
robust_results = {}

for metric_name, data in key_metrics.items():
    if len(data) >= 3:
        bootstrap_stats, ci_lower, ci_upper = robust_bootstrap_ci(data.values)
        observed = data.mean()

        robust_results[metric_name] = {
            'mean': observed,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'margin_error': (ci_upper - ci_lower) / 2,
            'n': len(data)
        }

        print(f"{metric_name}:")
        print(f"  📈 Media: {observed:.2f} [IC 95%: {ci_lower:.2f} - {ci_upper:.2f}]")
        print(f"  📊 Margen de error: ±{(ci_upper - ci_lower)/2:.2f} | n = {len(data)}")

# Análisis de confiabilidad del cuestionario (α de Cronbach)
def cronbach_alpha(items_df):
    """Calcula alfa de Cronbach para consistencia interna"""
    items_clean = items_df.dropna()
    if len(items_clean) < 3:
        return None

    k = items_clean.shape[1]
    item_variances = items_clean.var(axis=0, ddof=1)
    total_variance = items_clean.sum(axis=1).var(ddof=1)

    if total_variance == 0:
        return None

    alpha = (k / (k - 1)) * (1 - (item_variances.sum() / total_variance))
    return alpha

satisfaction_items = df[['satisfaccion_general', 'facilidad_uso', 'eficacia', 'claridad']]
alpha_reliability = cronbach_alpha(satisfaction_items)

print(f"\n🔬 CONFIABILIDAD DEL CUESTIONARIO:")
if alpha_reliability:
    print(f"α de Cronbach = {alpha_reliability:.3f}")
    if alpha_reliability > 0.9:
        reliability_level = "Excelente"
    elif alpha_reliability > 0.8:
        reliability_level = "Buena"
    elif alpha_reliability > 0.7:
        reliability_level = "Aceptable"
    else:
        reliability_level = "Baja"
    print(f"Interpretación: {reliability_level}")
else:
    print("No se pudo calcular (datos insuficientes)")


=== ANÁLISIS ESTADÍSTICO ROBUSTO ===

📊 MÉTRICAS PRINCIPALES CON INTERVALOS DE CONFIANZA (95%):
Tiempo Sin IA (min):
  📈 Media: 34.38 [IC 95%: 28.12 - 40.62]
  📊 Margen de error: ±6.25 | n = 8
Tiempo Con IA (min):
  📈 Media: 14.62 [IC 95%: 11.50 - 18.38]
  📊 Margen de error: ±3.44 | n = 8
Ahorro de Tiempo (min):
  📈 Media: 19.75 [IC 95%: 15.75 - 23.12]
  📊 Margen de error: ±3.69 | n = 8
Ahorro Porcentual (%):
  📈 Media: 57.57 [IC 95%: 53.23 - 62.20]
  📊 Margen de error: ±4.49 | n = 8
Satisfacción Total:
  📈 Media: 4.59 [IC 95%: 4.38 - 4.81]
  📊 Margen de error: ±0.22 | n = 8
Eficacia:
  📈 Media: 4.50 [IC 95%: 4.12 - 4.88]
  📊 Margen de error: ±0.38 | n = 8
Facilidad de Uso:
  📈 Media: 4.75 [IC 95%: 4.38 - 5.00]
  📊 Margen de error: ±0.31 | n = 8

🔬 CONFIABILIDAD DEL CUESTIONARIO:
α de Cronbach = 0.625
Interpretación: Baja


## Análisis Cualitativo

In [6]:
print("\n=== ANÁLISIS CUALITATIVO CON IA AVANZADO ===")

# Modelo de análisis de sentimiento más robusto
try:
    # Intentar modelo en español
    sentiment_classifier = pipeline("sentiment-analysis",
                                  model="pysentimiento/robertuito-sentiment-analysis")
    print("✅ Modelo de sentimientos en español cargado")
except:
    try:
        # Fallback a modelo multilingüe
        sentiment_classifier = pipeline("sentiment-analysis",
                                      model="nlptown/bert-base-multilingual-uncased-sentiment")
        print("✅ Modelo multilingüe cargado")
    except:
        # Fallback básico
        sentiment_classifier = pipeline("sentiment-analysis")
        print("✅ Modelo básico cargado")

# Análisis de sentimiento con confianza
def advanced_sentiment_analysis(text_series, column_name):
    """Análisis de sentimiento con métricas de confianza"""
    sentiments = []
    confidences = []

    for text in text_series:
        if pd.notna(text) and str(text).strip():
            try:
                result = sentiment_classifier(str(text))[0]
                sentiments.append(result['label'])
                confidences.append(result['score'])
            except:
                sentiments.append('NEUTRAL')
                confidences.append(0.5)
        else:
            sentiments.append('NO_RESPONSE')
            confidences.append(0.0)

    return sentiments, confidences

# Aplicar análisis de sentimiento mejorado
text_columns = ['gusto', 'desafios', 'mejoras', 'impacto_productividad']

for col in text_columns:
    if col in df.columns:
        sentiments, confidences = advanced_sentiment_analysis(df[col], col)
        df[f'sentimiento_{col}'] = sentiments
        df[f'confianza_{col}'] = confidences

# Resumen de sentimientos
print("\n📝 ANÁLISIS DE SENTIMIENTOS:")
for col in text_columns:
    if f'sentimiento_{col}' in df.columns:
        sent_counts = df[f'sentimiento_{col}'].value_counts()
        avg_confidence = df[f'confianza_{col}'].mean()

        print(f"\n{col.upper()}:")
        for sentiment, count in sent_counts.items():
            if sentiment != 'NO_RESPONSE':
                percentage = (count / len(df)) * 100
                print(f"  • {sentiment}: {count} respuestas ({percentage:.1f}%)")
        print(f"  • Confianza promedio: {avg_confidence:.2f}")

# Análisis de temas mejorado
def extract_advanced_themes(text_series, themes_dict):
    """Extracción de temas más sofisticada"""
    theme_results = []

    for text in text_series:
        if pd.isna(text) or not str(text).strip():
            theme_results.append('Sin_respuesta')
            continue

        text_lower = str(text).lower()
        found_themes = []

        for theme, keywords in themes_dict.items():
            if any(keyword in text_lower for keyword in keywords):
                found_themes.append(theme)

        if found_themes:
            theme_results.append(', '.join(found_themes))
        else:
            theme_results.append('Otro')

    return theme_results

# Diccionarios de temas más completos
positive_themes = {
    'Velocidad_Eficiencia': ['rápido', 'rápida', 'veloz', 'acelera', 'tiempo', 'eficiente', 'ahorro', 'pronto'],
    'Precisión_Calidad': ['preciso', 'precisa', 'exacto', 'correcto', 'calidad', 'acertado', 'confiable'],
    'Facilidad_Uso': ['fácil', 'simple', 'intuitivo', 'accesible', 'amigable', 'cómodo', 'sencillo'],
    'Centralización': ['centraliza', 'unifica', 'integra', 'lugar', 'sitio', 'organiza', 'concentra'],
    'Mejora_Productividad': ['productiv', 'eficaz', 'optimiza', 'mejora', 'potencia', 'aumenta']
}

challenge_themes = {
    'Limitaciones_Técnicas': ['limitado', 'falta', 'no puede', 'insuficiente', 'error', 'falla'],
    'Complejidad_Uso': ['complejo', 'complicado', 'difícil', 'confuso'],
    'Curva_Aprendizaje': ['aprender', 'acostumbrar', 'entrenar', 'adaptar', 'tiempo'],
    'Personalización': ['personalizar', 'adaptar', 'configurar', 'ajustar']
}

# Aplicar análisis de temas
df['temas_positivos'] = extract_advanced_themes(df['gusto'], positive_themes)
df['temas_desafios'] = extract_advanced_themes(df['desafios'], challenge_themes)

print(f"\n🎯 TEMAS PRINCIPALES IDENTIFICADOS:")
print(f"\nAspectos Positivos:")
positive_counts = df['temas_positivos'].value_counts()
for theme, count in positive_counts.head(5).items():
    if theme != 'Sin_respuesta':
        print(f"  • {theme.replace('_', ' ')}: {count} menciones")

print(f"\nDesafíos Identificados:")
challenge_counts = df['temas_desafios'].value_counts()
for theme, count in challenge_counts.head(5).items():
    if theme != 'Sin_respuesta':
        print(f"  • {theme.replace('_', ' ')}: {count} menciones")


=== ANÁLISIS CUALITATIVO CON IA AVANZADO ===


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

Device set to use cpu


✅ Modelo de sentimientos en español cargado

📝 ANÁLISIS DE SENTIMIENTOS:

GUSTO:
  • NEU: 6 respuestas (75.0%)
  • POS: 2 respuestas (25.0%)
  • Confianza promedio: 0.70

DESAFIOS:
  • NEU: 4 respuestas (50.0%)
  • NEG: 4 respuestas (50.0%)
  • Confianza promedio: 0.70

MEJORAS:
  • NEU: 8 respuestas (100.0%)
  • Confianza promedio: 0.82

IMPACTO_PRODUCTIVIDAD:
  • POS: 5 respuestas (62.5%)
  • NEU: 3 respuestas (37.5%)
  • Confianza promedio: 0.76

🎯 TEMAS PRINCIPALES IDENTIFICADOS:

Aspectos Positivos:
  • Otro: 3 menciones
  • Velocidad Eficiencia: 2 menciones
  • Precisión Calidad: 1 menciones
  • Centralización: 1 menciones
  • Velocidad Eficiencia, Centralización: 1 menciones

Desafíos Identificados:
  • Otro: 6 menciones
  • Limitaciones Técnicas: 2 menciones


## Clustering con K-Means



In [7]:
print("\n=== ANÁLISIS DE ARQUETIPOS DE USUARIO ===")

# Preparar datos para clustering
cluster_features = ['porcentaje_ahorro', 'satisfaccion_total', 'eficacia', 'facilidad_uso']
X_cluster = df[cluster_features].dropna()

if len(X_cluster) >= 3:
    # Estandarización
    scaler = StandardScaler()
    X_cluster_scaled = scaler.fit_transform(X_cluster)

    # Encontrar número óptimo de clusters
    max_clusters = min(4, len(X_cluster) - 1)
    if max_clusters >= 2:
        silhouette_scores = []
        cluster_range = range(2, max_clusters + 1)

        for k in cluster_range:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            cluster_labels = kmeans.fit_predict(X_cluster_scaled)
            if len(set(cluster_labels)) > 1:
                score = silhouette_score(X_cluster_scaled, cluster_labels)
                silhouette_scores.append(score)
            else:
                silhouette_scores.append(-1)

        if silhouette_scores:
            optimal_k = cluster_range[np.argmax(silhouette_scores)]
            best_score = max(silhouette_scores)

            print(f"📊 Clustering Optimizado:")
            print(f"• Número óptimo de clusters: {optimal_k}")
            print(f"• Puntuación Silhouette: {best_score:.3f}")

            # Aplicar clustering final
            kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
            df.loc[X_cluster.index, 'cluster'] = kmeans_final.fit_predict(X_cluster_scaled)

            # Caracterizar clusters (Arquetipos)
            print(f"\n👥 ARQUETIPOS DE USUARIO IDENTIFICADOS:")

            cluster_profiles = df.groupby('cluster').agg({
                'porcentaje_ahorro': 'mean',
                'satisfaccion_total': 'mean',
                'eficacia': 'mean',
                'facilidad_uso': 'mean',
                'rol': lambda x: ', '.join(x.dropna().astype(str))
            }).round(2)

            # Nombrar arquetipos basado en características
            archetype_names = {}
            for cluster_id, row in cluster_profiles.iterrows():
                if row['satisfaccion_total'] >= 4.0 and row['porcentaje_ahorro'] >= 20:
                    archetype_names[cluster_id] = "🌟 Súper Usuario"
                elif row['satisfaccion_total'] >= 3.5:
                    archetype_names[cluster_id] = "😊 Usuario Satisfecho"
                elif row['porcentaje_ahorro'] >= 15:
                    archetype_names[cluster_id] = "⚡ Usuario Eficiente"
                else:
                    archetype_names[cluster_id] = "🤔 Usuario Escéptico"

            for cluster_id, row in cluster_profiles.iterrows():
                archetype = archetype_names.get(cluster_id, f"Cluster {cluster_id}")
                cluster_size = (df['cluster'] == cluster_id).sum()

                print(f"\n{archetype} ({cluster_size} usuarios):")
                print(f"  • Ahorro promedio: {row['porcentaje_ahorro']:.1f}%")
                print(f"  • Satisfacción: {row['satisfaccion_total']:.1f}/5.0")
                print(f"  • Eficacia percibida: {row['eficacia']:.1f}/5.0")
                print(f"  • Roles: {row['rol']}")


=== ANÁLISIS DE ARQUETIPOS DE USUARIO ===
📊 Clustering Optimizado:
• Número óptimo de clusters: 3
• Puntuación Silhouette: 0.391

👥 ARQUETIPOS DE USUARIO IDENTIFICADOS:

🌟 Súper Usuario (2 usuarios):
  • Ahorro promedio: 50.0%
  • Satisfacción: 4.4/5.0
  • Eficacia percibida: 4.5/5.0
  • Roles: Desarrollador ABAP Senior, Desarrollador ABAP Senior

🌟 Súper Usuario (3 usuarios):
  • Ahorro promedio: 57.2%
  • Satisfacción: 4.4/5.0
  • Eficacia percibida: 4.0/5.0
  • Roles: Desarrollador ABAP Junior, Desarrollador ABAP Senior, Desarrollador ABAP Senior

🌟 Súper Usuario (3 usuarios):
  • Ahorro promedio: 63.0%
  • Satisfacción: 4.9/5.0
  • Eficacia percibida: 5.0/5.0
  • Roles: Desarrollador ABAP Junior, Desarrollador ABAP Senior, Gerente de Proyectos TI


## Visualización


In [8]:
print("\n=== GENERANDO VISUALIZACIONES AVANZADAS ===")

# 1. Dashboard principal mejorado
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribución de Métricas Clave', 'Correlación Ahorro-Satisfacción',
                   'Perfiles por Rol', 'Arquetipos de Usuario'),
    specs=[[{"secondary_y": True}, {"type": "scatter"}],
           [{"type": "box"}, {"type": "scatter"}]]
)

# Gráfico 1: Métricas con intervalos de confianza
metrics_for_viz = ['Tiempo Sin IA (min)', 'Tiempo Con IA (min)', 'Satisfacción Total']
colors = ['lightcoral', 'lightblue', 'lightgreen']

for i, (metric, color) in enumerate(zip(metrics_for_viz, colors)):
    if metric in robust_results:
        result = robust_results[metric]
        fig.add_trace(
            go.Bar(
                x=[metric.replace(' (min)', '').replace('Satisfacción Total', 'Satisfacción')],
                y=[result['mean']],
                error_y=dict(type='data', array=[result['margin_error']], visible=True),
                name=f'{metric}',
                marker_color=color,
                showlegend=False
            ),
            row=1, col=1
        )

# Gráfico 2: Scatter con tamaño por eficacia
fig.add_trace(
    go.Scatter(
        x=df['porcentaje_ahorro'],
        y=df['satisfaccion_total'],
        mode='markers+text',
        text=df['rol'],
        textposition="top center",
        marker=dict(
            size=df['eficacia'] * 8,
            color=df['facilidad_uso'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Facilidad de Uso", x=1.02)
        ),
        name='Usuarios',
        showlegend=False
    ),
    row=1, col=2
)

# Gráfico 3: Box plots por rol
roles_unicos = df['rol'].dropna().unique()
for rol in roles_unicos:
    fig.add_trace(
        go.Box(
            y=df[df['rol'] == rol]['satisfaccion_total'],
            name=str(rol),
            showlegend=False
        ),
        row=2, col=1
    )

# Gráfico 4: Clusters si existen
if 'cluster' in df.columns:
    cluster_colors = px.colors.qualitative.Set1
    unique_clusters = df['cluster'].dropna().unique()

    for i, cluster in enumerate(unique_clusters):
        cluster_data = df[df['cluster'] == cluster]
        archetype_name = archetype_names.get(cluster, f"Cluster {cluster}")

        fig.add_trace(
            go.Scatter(
                x=cluster_data['porcentaje_ahorro'],
                y=cluster_data['satisfaccion_total'],
                mode='markers',
                marker=dict(
                    size=15,
                    color=cluster_colors[i % len(cluster_colors)],
                    line=dict(width=2, color='white')
                ),
                name=archetype_name,
                showlegend=True
            ),
            row=2, col=2
        )

# Actualizar layout
fig.update_layout(
    height=800,
    title_text="🚀 Dashboard Avanzado de Análisis de IA",
    showlegend=True
)

# Actualizar ejes
fig.update_xaxes(title_text="% Ahorro de Tiempo", row=1, col=2)
fig.update_yaxes(title_text="Satisfacción Total", row=1, col=2)
fig.update_xaxes(title_text="% Ahorro de Tiempo", row=2, col=2)
fig.update_yaxes(title_text="Satisfacción Total", row=2, col=2)

fig.show()

# 2. Radar Chart de perfiles de usuario
print("\n📊 Generando Radar Chart de Perfiles...")

if len(df) > 0:
    categories = ['Satisfacción General', 'Facilidad Uso', 'Eficacia', 'Claridad']
    fig_radar = go.Figure()

    colors = px.colors.qualitative.Set1

    for i, (idx, row) in enumerate(df.iterrows()):
        if pd.notna(row['rol']):
            values = [
                row['satisfaccion_general'],
                row['facilidad_uso'],
                row['eficacia'],
                row['claridad']
            ]

            # Solo mostrar si no hay valores nulos
            if not any(pd.isna(values)):
                fig_radar.add_trace(go.Scatterpolar(
                    r=values,
                    theta=categories,
                    fill='toself',
                    name=f"{row['rol']}",
                    line_color=colors[i % len(colors)],
                    opacity=0.7
                ))

    fig_radar.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 5]
            )
        ),
        showlegend=True,
        title="🎯 Perfiles Multidimensionales de Usuarios",
        height=600
    )
    fig_radar.show()


=== GENERANDO VISUALIZACIONES AVANZADAS ===



📊 Generando Radar Chart de Perfiles...
